In [ ]:
# Chatbot with Gemini, Model Armor, and Prompt Filtering

from vertexai.preview.language_models import ChatModel, ChatMessage
from vertexai.preview.model_garden import ModelGarden
from google.api_core.client_options import ClientOptions
from google.cloud import modelarmor_v1
import re

# ─────────────────────────────────────────────
# CONFIGURATION
# ─────────────────────────────────────────────
project_id = "qwiklabs-gcp-00-711e7fbdb4c8"
location_id = "us-central1"
template_id = "hypothetical_template_id"


# ─────────────────────────────────────────────
# MODEL ARMOR TEMPLATE SETUP
# ─────────────────────────────────────────────

# Create the Model Armor client.
client = modelarmor_v1.ModelArmorClient(
    transport="rest",
    client_options=ClientOptions(
        api_endpoint=f"modelarmor.{location_id}.rep.googleapis.com"
    ),
)

# Build the Model Armor template
template = modelarmor_v1.Template(
    filter_config=modelarmor_v1.FilterConfig(
        pi_and_jailbreak_filter_settings=modelarmor_v1.PiAndJailbreakFilterSettings(
            filter_enforcement=modelarmor_v1.PiAndJailbreakFilterSettings.PiAndJailbreakFilterEnforcement.ENABLED,
            confidence_level=modelarmor_v1.DetectionConfidenceLevel.MEDIUM_AND_ABOVE,
        ),
        malicious_uri_filter_settings=modelarmor_v1.MaliciousUriFilterSettings(
            filter_enforcement=modelarmor_v1.MaliciousUriFilterSettings.MaliciousUriFilterEnforcement.ENABLED,
        ),
    ),
)

# Prepare the request for creating the template.
request = modelarmor_v1.CreateTemplateRequest(
    parent=f"projects/{project_id}/locations/{location_id}",
    template_id=hypothetical_template_id,
    template=template,
)

# Create the template.
response = client.create_template(request=request)

# ─────────────────────────────────────────────
# GEMINI MODEL INITIALIZATION
# ─────────────────────────────────────────────
ModelGarden.configure(project="qwiklabs-gcp-00-711e7fbdb4c8")
chat_model = ChatModel.from_pretrained("gemini-pro")


# ─────────────────────────────────────────────
# CHATBOT GOALS & RESTRICTIONS
# ─────────────────────────────────────────────
goals = [
    "Provide helpful and informative responses.",
    "Maintain a friendly and engaging tone.",
    "You are a programming chat bot answering coding questions using Python unless another language is specified.",
    "Format code according to the Python PEP 8 Style guide."
    ]
restrictions = [
    "Do not answer questions unrelated to Computers or Programming.",
    "For any questions unrelated to Computers or Programming, just say you cannot help with that.",
    "Do not generate responses that are offensive or harmful."
    ]

# ─────────────────────────────────────────────
# MODEL ARMOR INPUT CHECK
# ─────────────────────────────────────────────
def model_armor_check(response_text):

  user_prompt = response_text

  # Create the Model Armor client.
  client = modelarmor_v1.ModelArmorClient(
      transport="rest",
      client_options=ClientOptions(
          api_endpoint=f"modelarmor.{location_id}.rep.googleapis.com"
      ),
  )

  # Initialize request argument(s).
  user_prompt_data = modelarmor_v1.DataItem(text=user_prompt)

  # Prepare request for sanitizing the defined prompt.
  request = modelarmor_v1.SanitizeUserPromptRequest(
      name=f"projects/{project_id}/locations/{location_id}/templates/{hypothetical_template_id}",
      user_prompt_data=user_prompt_data,
  )

  # Sanitize the user prompt.
  response = client.sanitize_user_prompt(request=request)
  return response

# ─────────────────────────────────────────────
# PROMPT FILTERING
# ─────────────────────────────────────────────
prompt_filters = [
    r"\b(bomb|kill|explosive|harm|weapon)\b", # Filter specific bad words
]

# Function for prompt filtering
def filter_prompt(user_input):
    """Applies regex and Model Armor filters to user input."""
  for pattern in prompt_filters:
    if re.search(pattern, user_input, re.IGNORECASE):
      return "Your input contains inappropriate content. Please try again."

    if model_armor_check(user_input).filter_match_state != 'MATCH_FOUND':
      return "Your input contains inappropriate content. Please try again."

  return user_input

# ─────────────────────────────────────────────
# RESPONSE FILTERING
# ─────────────────────────────────────────────
def filter_response(response_text):

  # Armor Check
  armor_check_result = model_armor_check(response_text).filter_match_state

  if armor_check_result == 'MATCH_FOUND':
    return "I'm unable to provide a response to that request."
  return response_text


# ─────────────────────────────────────────────
# CHAT LOOP
# ─────────────────────────────────────────────
while True:
  user_input = input("You: ")

  if user_input.lower() in ["exit", "quit"]:
    print("Chatbot: Goodbye")
    break

  # Step 1: Filter Prompt
  filtered_prompt = filter_prompt(user_input)

  if filtered_prompt != user_input:
    print("Chatbot:", filtered_prompt)
    continue


  # Step 2: Send to Gemini
  messages = [ChatMessage(content=filtered_prompt, author="user")]
  response = chat_model.start_chat(
      context=f"Goals: {', '.join(goals)}\nRestrictions: {', '.join(restrictions)}",
      messages=messages
  ).send_message()

  # Step 3: Filter model response
  filtered_response = filter_response(response.text)
  print("Chatbot:", filtered_response)

